In [1]:
import numpy as np
import pandas as pd

In [56]:
import numbers

In [66]:
import math

In [71]:
from operator import itemgetter

In [74]:
import time

### Data Preprocessing

In [3]:
df_300 = pd.read_csv("../../PEW_Jun29_300.csv")

In [4]:
df_300

,Worker ID,Saving for the future,Paying for college,Buying a home,Finding a spouse or partner,Finding a job,Getting into college,Staying in touch with family and friends,Do you currently live in the United States?,What is your age?
0,A2MS1GQLGAX9FZ,Harder,Harder,Harder,Harder,Harder,Easier,Easier,Yes,24.0
1,A211TN268BD80Q,Easier,Easier,Easier,Easier,Easier,Easier,Easier,Yes,4.0
2,A172PA5HBQNKQM,Harder,Same,Easier,Harder,Easier,Easier,Harder,Yes,25.0
3,AKZ3J53DA84EX,Easier,Easier,Easier,Easier,Easier,Easier,Easier,Yes,30.0
4,A3IOAOY6RR12YN,Easier,Easier,Easier,Harder,Easier,Easier,Easier,No,45.0
...,...,...,...,...,...,...,...,...,...,...
295,A7AXZSG0CTRLG,Same,Harder,Harder,Same,Same,Harder,Same,Yes,32.0
296,A172SQXS2LLR43,Harder,Easier,Easier,Easier,Harder,Harder,Easier,Yes,44.0
297,AWWBF2RVU8HNM,Easier,Easier,Easier,Easier,Easier,Easier,Easier,Yes,37.0
298,A3335ASFR3L0Z8,Harder,Harder,Harder,Harder,Harder,Same,Harder,Yes,33.0


In [5]:
question_types = ["radio", "radio", "radio", "radio", "radio", "radio", "radio", "radio", "estimate"]

In [22]:
c_age = df_300['What is your age?']
radix = [3, 3, 3, 3, 3, 3, 3, 2, int(c_age.max()-c_age.min())]

In [25]:
import random

In [28]:
# n is number of samples to create
# radixes describes what a sample should look like: possibilities for each sample
def createSamples(n, radixes):

    samples = []
    length = len(radixes)

    for x in range(n):

        sample = []

        for y in range(length):

            # for each item, randomly choose using probabilities
            radix = radixes[y]

            newNumber = random.randint(0, radix-1)
            sample.append(newNumber)

        samples.append(sample)


    return np.array(samples)

In [32]:
sample_random = createSamples(300,radix)
sample_random

array([[ 0,  0,  1, ...,  1,  1, 33],
       [ 1,  1,  2, ...,  1,  0,  3],
       [ 1,  1,  0, ...,  1,  0, 27],
       ...,
       [ 2,  0,  2, ...,  1,  0, 60],
       [ 2,  0,  0, ...,  1,  1, 45],
       [ 1,  1,  2, ...,  2,  1, 19]])

In [61]:
sample_300 = np.array(df_300.drop('Worker ID', axis=1).applymap(
    # convert answers to integers (within radix)
    lambda x: {
        # dictionary for radio question
        'Easier': 0,
        'Same': 1,
        'Harder': 2,
        # dictionary for boolean radio
        'Yes': 1,
        'No': 0,
    }.get(x, int(x) if isinstance(x, numbers.Number) else x) # if an estimate, make it an int number
))
sample_300

array([[ 2,  2,  2, ...,  0,  1, 24],
       [ 0,  0,  0, ...,  0,  1,  4],
       [ 2,  1,  0, ...,  2,  1, 25],
       ...,
       [ 0,  0,  0, ...,  0,  1, 37],
       [ 2,  2,  2, ...,  2,  1, 33],
       [ 0,  2,  0, ...,  0,  1, 39]])

### Measuring Performance of <ANONYMIZED>'s code

In [98]:
def earth_movers_1(samples1, samples2, radixes, question_types):

    if (len(samples1) != len(samples2)):
        print("Error! Lists should be the same length!")
        return

    if (len(samples1[0]) != len(samples2[0])):
        print("Error! Each item in each list should be the same length!")
        return

    numberOfSamples = len(samples1)
    complexity = len(radixes)

    # data is stored in this format:
    # [x1y1, x1y2, x1y3 ... x2y1, x2y2...] where x1y2 is the distance between element 1 of list 1 and element 1 of list 2

    data = []

    # exhaustively compute the distance between every pair
    for x in range(numberOfSamples):

        for y in range(numberOfSamples):

            s1 = samples1[x]
            s2 = samples2[y]

            total = 0


            # Calculate euclidean distance
            for z in range(complexity):

                # check the question type
                if question_types[z] == "radio" or question_types[z] == "checkbox":

                    # for radio and checkbox questions, different is 1 and same is 0
                    if s1[z] != s2[z]:
                        total += 1

                else:
                    # Right now the only other case is estimate. For these, normalize between 0 and 1
                    radix = radixes[z]
                    percent1 = s1[z] / (radix - 1)
                    percent2 = s2[z] / (radix - 1)
                    diff = percent1 - percent2
                    total += (diff * diff)

            # Append the distance, plus the x and y
            totalRoot = math.sqrt(total)
            data.append([x,y,totalRoot])

    return data


def earth_movers_2(data):

    distance = 0

    # sort
    sortedData = sorted(data, key=itemgetter(2))

    return sortedData


def earth_movers_3(sortedData, numberOfSamples):
    # rows and columns already used
    rowsUsed = []
    columnsUsed = []

    number_chosen = 0
    distance = 0.0
    index = 0

    # go through and find the smallest value until every item has been "moved"
    while (number_chosen < numberOfSamples):

        current = sortedData[index]
        row = current[0]
        column = current[1]
        dist = current[2]

        if row not in rowsUsed and column not in columnsUsed:

            distance += dist
            number_chosen += 1
            rowsUsed.append(row)
            columnsUsed.append(column)

        index += 1

    return distance


In [101]:
tic = time.perf_counter()

for i in range(0,100):
    print(i, end=' ')
    earth_movers_1(sample_random, sample_300, radix, question_types)

toc = time.perf_counter()
print(f"earth_movers_1: time taken: {toc - tic:0.4f} seconds")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 earth_movers_1: time taken: 23.5659 seconds


In [96]:
distance_matrix = earth_movers_1(sample_random, sample_300, radix, question_types)
tic = time.perf_counter()

for i in range(0,100):
    print(i, end=' ')
    earth_movers_2(distance_matrix)

toc = time.perf_counter()
print(f"earth_movers_2: time taken: {toc - tic:0.4f} seconds")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 earth_movers_2: time taken: 1.0490 seconds


In [100]:
sorted_distance_matrix = earth_movers_2(distance_matrix)

tic = time.perf_counter()

for i in range(0,100):
    print(i, end=' ')
    earth_movers_3(sorted_distance_matrix, len(sample_300))

toc = time.perf_counter()
print(f"earth_movers_2: time taken: {toc - tic:0.4f} seconds")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 earth_movers_2: time taken: 9.3295 seconds


In [81]:
tic = time.perf_counter()

for i in range(0,100):
    print(i, earth_movers(sample_random, sample_300, radix, question_types))

toc = time.perf_counter()
print(f"Time taken: {toc - tic:0.4f} seconds")

0 477.24664109137547
1 477.24664109137547
2 477.24664109137547
3 477.24664109137547
4 477.24664109137547
5 477.24664109137547
6 477.24664109137547
7 477.24664109137547
8 477.24664109137547
9 477.24664109137547
10 477.24664109137547
11 477.24664109137547
12 477.24664109137547
13 477.24664109137547
14 477.24664109137547
15 477.24664109137547
16 477.24664109137547
17 477.24664109137547
18 477.24664109137547
19 477.24664109137547
20 477.24664109137547
21 477.24664109137547
22 477.24664109137547
23 477.24664109137547
24 477.24664109137547
25 477.24664109137547
26 477.24664109137547
27 477.24664109137547
28 477.24664109137547
29 477.24664109137547
30 477.24664109137547
31 477.24664109137547
32 477.24664109137547
33 477.24664109137547
34 477.24664109137547
35 477.24664109137547
36 477.24664109137547
37 477.24664109137547
38 477.24664109137547
39 477.24664109137547
40 477.24664109137547
41 477.24664109137547
42 477.24664109137547
43 477.24664109137547
44 477.24664109137547
45 477.2466410913754

In [82]:
list_random = sample_random.tolist()
list_300 = sample_300.tolist()

tic = time.perf_counter()

for i in range(0,100):
    print(i, earth_movers(list_random, list_300, radix, question_types))

toc = time.perf_counter()
print(f"Time taken: {toc - tic:0.4f} seconds")

0 477.24664109137547
1 477.24664109137547
2 477.24664109137547
3 477.24664109137547
4 477.24664109137547
5 477.24664109137547
6 477.24664109137547
7 477.24664109137547
8 477.24664109137547
9 477.24664109137547
10 477.24664109137547
11 477.24664109137547
12 477.24664109137547
13 477.24664109137547
14 477.24664109137547
15 477.24664109137547
16 477.24664109137547
17 477.24664109137547
18 477.24664109137547
19 477.24664109137547
20 477.24664109137547
21 477.24664109137547
22 477.24664109137547
23 477.24664109137547
24 477.24664109137547
25 477.24664109137547
26 477.24664109137547
27 477.24664109137547
28 477.24664109137547
29 477.24664109137547
30 477.24664109137547
31 477.24664109137547
32 477.24664109137547
33 477.24664109137547
34 477.24664109137547
35 477.24664109137547
36 477.24664109137547
37 477.24664109137547
38 477.24664109137547
39 477.24664109137547
40 477.24664109137547
41 477.24664109137547
42 477.24664109137547
43 477.24664109137547
44 477.24664109137547
45 477.2466410913754

### Calculate distance matrix

In [85]:
np.bincount(sample_300[:,0])

array([ 93,  31, 176])

In [86]:
np.bincount(sample_random[:,0])

array([101,  99, 100])

In [118]:
s1, s2 = sample_random, sample_300

s1.shape == s2.shape

True

In [139]:
# vectorized
distance = np.zeros((sample_random.shape[0], sample_random.shape[0]))

for z in range(0, 9):
    # the below works for radio and checkbox
    if question_types[z] == "radio" or question_types[z] == "checkbox":
        # TODO: further optimize with memoization
        for x in range(0, sample_random.shape[0]):
            distance[x] += np.where(sample_300[:,z] == sample_random[x][z], 0, 1)

    # what about estimate questions?
    else:
        rad = radix[z]

        for x in range(0, sample_random.shape[0]):
            distance[x] += np.abs((sample_300[:,z] - sample_random[x][z])/rad)

distance

array([[6.13846154, 4.44615385, 6.12307692, ..., 4.06153846, 7.        ,
        6.09230769],
       [6.32307692, 7.01538462, 5.33846154, ..., 7.52307692, 6.46153846,
        7.55384615],
       [6.04615385, 6.35384615, 4.03076923, ..., 6.15384615, 7.09230769,
        7.18461538],
       ...,
       [6.55384615, 6.86153846, 6.53846154, ..., 6.35384615, 5.41538462,
        8.32307692],
       [6.32307692, 3.63076923, 4.30769231, ..., 3.12307692, 5.18461538,
        5.09230769],
       [5.07692308, 7.23076923, 5.09230769, ..., 7.27692308, 3.21538462,
        6.30769231]])

In [140]:
# modified original
distance_alt = np.zeros((sample_random.shape[0], sample_random.shape[0]))

for x in range(0, sample_random.shape[0]):
    for y in range(0, sample_300.shape[0]):

        s1 = sample_random[x]
        s2 = sample_300[y]

        total = 0

        # Calculate euclidean distance
        for z in range(0, 9): # excluding

            # check the question type
            if question_types[z] == "radio" or question_types[z] == "checkbox":

                # for radio and checkbox questions, different is 1 and same is 0
                if s1[z] != s2[z]:
                    total += 1

            else:
                rad = radix[z]
                percent1 = s1[z] / (rad)
                percent2 = s2[z] / (rad)
                diff = percent1 - percent2
                total += abs(diff)


        # Append the distance, plus the x and y
        # totalRoot = math.sqrt(total)
        # data.append([x,y,totalRoot])

        distance_alt[x][y] = total

distance_alt

array([[6.13846154, 4.44615385, 6.12307692, ..., 4.06153846, 7.        ,
        6.09230769],
       [6.32307692, 7.01538462, 5.33846154, ..., 7.52307692, 6.46153846,
        7.55384615],
       [6.04615385, 6.35384615, 4.03076923, ..., 6.15384615, 7.09230769,
        7.18461538],
       ...,
       [6.55384615, 6.86153846, 6.53846154, ..., 6.35384615, 5.41538462,
        8.32307692],
       [6.32307692, 3.63076923, 4.30769231, ..., 3.12307692, 5.18461538,
        5.09230769],
       [5.07692308, 7.23076923, 5.09230769, ..., 7.27692308, 3.21538462,
        6.30769231]])

### Finding minimum distance

In [90]:
import lap

In [142]:
lap.lapjv(distance)

(820.876923076923,
 array([133,  52,  96,  70,   8, 196, 229,  95,  90,   9, 187,  54, 205,
          1, 192, 226, 106,  43,  11, 117, 120, 168, 107, 180, 291, 162,
        222, 146,  29,   3, 160, 213, 121, 250, 109,   5,  65, 188,  93,
        108, 264, 155, 169, 280, 242,  56, 101,  26,  67, 154, 238,  39,
        115, 266,  45, 253, 150, 254,  81, 113, 130,  63, 287, 260, 134,
        194,  55, 279, 225,   2,  74,  84,  12,  60, 140,  20, 203, 211,
         35,  42, 275,  92,  64,  51, 245, 230,  21, 152, 129,  37, 220,
        139, 186, 217,  15,   6, 179, 181, 123, 153, 111, 271,  82, 100,
        172, 236, 273, 219,  89,  48, 198, 215, 118,  69,  79, 274, 132,
        204, 138,  13, 257,  38,  66, 269, 221, 103, 159, 143, 201,  41,
        131, 149, 252, 234, 137,  86,  24, 212, 288, 285, 296, 227, 119,
        224,  27,  16, 290, 223,  19, 163, 270,  68,  87, 292, 102, 197,
        151, 135,  34, 214, 167, 105,  77, 193,  32, 156, 157, 202,  94,
          0, 246,  85,  99, 265,

## Vectorized Python Code

In [146]:
def earth_mover_distance(samples1: np.ndarray, samples2: np.ndarray, radixes, question_types):
    if (samples1.shape != samples2.shape):
        print("shape not the same!")
        return

    numberOfSamples = samples1.shape[0]
    numberOfQuestions = samples1.shape[1]

    if (numberOfQuestions != len(radixes)):
        print("radixes shape not match!")
        return

    if (numberOfQuestions != len(question_types)):
        print("question_types shape not match!")
        return

    ## Calculate distance matrix
    distance = np.zeros((numberOfSamples, numberOfSamples))

    for z in range(0, numberOfQuestions):
        # radio and checkbox: 1 if different; 0 if same
        if question_types[z] == "radio" or question_types[z] == "checkbox":
            # TODO: further optimize with memoization
            for x in range(0, samples1.shape[0]):
                distance[x] += np.where(samples2[:,z] == samples1[x][z], 0, 1)

        # estimate question: absolute difference / radix
        else:
            radix = radixes[z]
            for x in range(0, samples1.shape[0]):
                distance[x] += np.abs((samples2[:,z] - samples1[x][z])/radix)

    ## Find minimum distance matching
    return lap.lapjv(distance)[0]

In [150]:
tic = time.perf_counter()

for i in range(0,1000):
    print(i, end=' ')
    earth_mover_distance(sample_random, sample_300, radix, question_types)

toc = time.perf_counter()

print(f"\nVectorized: time taken: {toc - tic:0.4f} seconds")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [157]:
# column major
sample_random_f = np.array(sample_random, order='F')
sample_300_f = np.array(sample_300, order='F')

tic = time.perf_counter()

for i in range(0,1000):
    print(i, end=' ')
    earth_mover_distance(sample_random_f, sample_300_f, radix, question_types)

toc = time.perf_counter()

print(f"\nVectorized: time taken: {toc - tic:0.4f} seconds")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [151]:
tic = time.perf_counter()

for i in range(0,100):
    print(i, end=' ')
    earth_movers(sample_random, sample_300, radix, question_types)

toc = time.perf_counter()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [152]:
print(f"\nNaive: time taken: {toc - tic:0.4f} seconds")


Naive: time taken: 35.3395 seconds


In [161]:
sample_random.mean(axis=0)

array([ 0.99666667,  1.01      ,  0.95333333,  1.07333333,  0.89666667,
        0.93333333,  0.98333333,  0.50333333, 34.98333333])

## Vectorized & Memoized

In [178]:
"""
"""
def earth_mover_distance_memo(samples1: np.ndarray, samples2: np.ndarray, radixes, question_types):
    if (samples1.shape != samples2.shape):
        print("shape not the same!")
        return

    numberOfSamples = samples1.shape[0]
    numberOfQuestions = samples1.shape[1]

    if (numberOfQuestions != len(radixes)):
        print("radixes shape not match!")
        return

    if (numberOfQuestions != len(question_types)):
        print("question_types shape not match!")
        return

    ## Calculate distance matrix
    distance = np.zeros((numberOfSamples, numberOfSamples))
    memoMap = {}  # (z, valueInX) => np.ndarray of cost

    for z in range(0, numberOfQuestions):
        # radio and checkbox: 1 if different; 0 if same
        if question_types[z] == "radio" or question_types[z] == "checkbox":
            for x in range(0, samples1.shape[0]):  # TODO: probably parallelize this?
                value = samples1[x][z]  # value of samples1 in this col
                if (z,value) in memoMap.keys():
                    distance[x] += memoMap[(z,value)]
                else:
                    memo = np.where(samples2[:,z] == value, 0, 1)
                    memoMap[(z,value)] = memo
                    distance[x] += memo

        # estimate question: absolute difference / radix
        else:
            radix = radixes[z]
            for x in range(0, samples1.shape[0]):
                value = samples1[x][z]
                if (z,value) in memoMap.keys():
                    distance[x] += memoMap[(z,value)]
                else:
                    memo = np.abs((samples2[:,z] - value)/radix)
                    memoMap[(z,value)] = memo
                    distance[x] += memo

    ## Find minimum distance matching
    return lap.lapjv(distance)[0]

In [179]:
tic = time.perf_counter()

for i in range(0,1000):
    # print(i, end=' ')
    earth_mover_distance_memo(sample_random_f, sample_300_f, radix, question_types)

toc = time.perf_counter()

print(f"\nVectorized: time taken: {toc - tic:0.4f} seconds")


Vectorized: time taken: 5.8636 seconds


In [166]:
earth_mover_distance_memo(sample_random_f, sample_300_f, radix, question_types)

820.876923076923

In [168]:
earth_mover_distance(sample_random, sample_300, radix, question_types)

820.876923076923

In [175]:
tic = time.perf_counter()

for i in range(0,1000):
    # print(i, end=' ')
    createSamples(300, radix)

toc = time.perf_counter()

print(f"\nVectorized: time taken: {toc - tic:0.4f} seconds")


Vectorized: time taken: 1.5375 seconds


## Final code (with comments)

In [ ]:
from typing import List
import numpy as np
import lap

def earth_mover_distance_memo(samples1: np.ndarray, samples2: np.ndarray, radixes: List[int], question_types: List[str]):
    """Optimized method for calculating Earth Mover Distance between two Survey response samples.

    This method is vectorized and memoized.

    Args:
        samples1 (np.ndarray): The first sample of survey response. Shape should be N*Q.
        samples2 (np.ndarray): The second sample of survey response. Shape should be N*Q.
        radixes (List[int]): A list of the radixes used to respectively represent survey questions. Should be length Q
        question_types (List[str]): A list of the survey question types. Used to figure distance calculation algorithm. Should be length Q.

    Returns:
        _type_: _description_
    """

    ## Sanity check
    if (samples1.shape != samples2.shape):
        print("samples1 and samples2 shape not the same!")
        return

    # N: number of samples in both samples1 and samples2
    numberOfSamples = samples1.shape[0]
    # Q: number of questions in the survey
    numberOfQuestions = samples1.shape[1]

    if (numberOfQuestions != len(radixes)):
        print("radixes length not match!")
        return

    if (numberOfQuestions != len(question_types)):
        print("question_types length not match!")
        return

    ## Calculating distance matrix
    # distance matrix (shape N*N). Element distance[x][y] denotes distance between x-th element in samples1 and y-th element in samples2
    distance = np.zeros((numberOfSamples, numberOfSamples))
    # Memoizes and eliminates unnecessary computations. (z, valueInX) => np.ndarray of cost
    memoMap = {}

    for z in range(0, numberOfQuestions):
        # radio and checkbox question: distance = 1 if different else 0
        if question_types[z] == "radio" or question_types[z] == "checkbox":
            # TODO: probably the following loop can be parallelized?
            for x in range(0, samples1.shape[0]):
                value = samples1[x][z]  # value of samples1 in this col

                # Note that we can memoize the operation here: for the same value, the cost contribution of *this question* will always be the same when comparing across entire samples2
                if (z, value) in memoMap.keys():
                    distance[x] += memoMap[(z, value)]
                else:
                    # Note that this operation is vectorized: numpy has builtin function for comparing & setting values
                    memo = np.where(samples2[:, z] == value, 0, 1)
                    memoMap[(z, value)] = memo
                    distance[x] += memo

        # estimate question: distance = absolute difference / radix
        else:
            radix = radixes[z]
            for x in range(0, samples1.shape[0]):
                value = samples1[x][z]
                if (z, value) in memoMap.keys():
                    distance[x] += memoMap[(z, value)]
                else:
                    # Note that this operation is vectorized: numpy has builtin function for subtracting and division across entire row
                    memo = np.abs((samples2[:, z] - value)/radix)
                    memoMap[(z, value)] = memo
                    distance[x] += memo

    ## Find minimum distance matching (i.e. EMD) with Jonker–Volgenant algorithm
    return lap.lapjv(distance)[0]
